In [16]:
import pandas as pd

# Sample data
data = {
    'ticket_id': [1, 2, 3, 4, 5, 6, 7],
    'text': [
        'I need to know the price of the custom necklace.',
        'My order number 1234 has not been delivered.',
        'Can I get a discount on bulk orders?',
        'The bracelet I received is broken.',
        'I want to return my order and get a refund.',
        'How much does it cost to ship to Canada?',
        'I am interested in a custom-designed ring.'
    ],
    'category': ['pricing', 'delivery_issue', 'discount', 'product_issue', 'refund', 'shipping', 'custom_order'],
    'amount_involved': [50, 10000, 500, 75, 150000, 20, 300]  # Example monetary values
}

# Create DataFrame
df = pd.DataFrame(data)


In [17]:
import re
import string

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()  # Remove whitespace
    return text

# Apply preprocessing
df['clean_text'] = df['text'].apply(preprocess_text)


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['category'], test_size=0.2, random_state=42)

# Define a pipeline combining a TF-IDF vectorizer with a logistic regression classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on test data
y_pred = pipeline.predict(X_test)




In [24]:
def assign_priority(amount):
    if amount >=10000:
        return 'high'

    else:
        return 'low'


In [25]:
def route_ticket(ticket):
    text = ticket['clean_text']
    category = pipeline.predict([text])[0]
    amount = ticket['amount_involved']
    priority = assign_priority(amount)

    if priority == 'high':
        return 'main_admin'
    else:
        return 'sub_admin'

# Route tickets
df['assigned_to'] = df.apply(route_ticket, axis=1)

# Show routing results
print(df[['ticket_id', 'text', 'category', 'amount_involved', 'assigned_to']])


   ticket_id                                              text  \
0          1  I need to know the price of the custom necklace.   
1          2      My order number 1234 has not been delivered.   
2          3              Can I get a discount on bulk orders?   
3          4                The bracelet I received is broken.   
4          5       I want to return my order and get a refund.   
5          6          How much does it cost to ship to Canada?   
6          7        I am interested in a custom-designed ring.   

         category  amount_involved assigned_to  
0         pricing               50   sub_admin  
1  delivery_issue            10000  main_admin  
2        discount              500   sub_admin  
3   product_issue               75   sub_admin  
4          refund           150000  main_admin  
5        shipping               20   sub_admin  
6    custom_order              300   sub_admin  
